In [1]:
# convert_coco.py
from ultralytics.data.converter import convert_coco

# point this at your COCO JSON folder:
convert_coco(
    labels_dir="../data/cad/annotations",  # contains your train.json & val.json
    save_dir="../data/cad",                # will create labels/train & labels/val
    use_segments=True,                           # include mask polygons
    cls91to80=False                              # our dataset only has 16 classes
)


Annotations /home/ubuntu/floor-plan-net/data/cad/annotations/test-00-annotations.json: 100%|██████████| 3893/3893 [00:00<00:00, 7921.33it/s]
Annotations /home/ubuntu/floor-plan-net/data/cad/annotations/train-00-annotations.json: 100%|██████████| 2903/2903 [00:00<00:00, 7626.46it/s]
Annotations /home/ubuntu/floor-plan-net/data/cad/annotations/train-01-annotations.json: 100%|██████████| 4336/4336 [00:00<00:00, 8961.55it/s]

COCO data converted successfully.
Results saved to /home/ubuntu/floor-plan-net/data/cad3


In [8]:
from ultralytics import YOLO

# load your segmentation-capable checkpoint
model = YOLO("models/yolo11l-seg.pt")

# train
model.train(
    data="data/cad/cad.yaml",
    epochs=20,
    batch=16,
    imgsz=640,
    device=0,       # GPU 0 (or "cpu")
    lr0=2e-3,
    freeze=0,
    name="cad-seg-run"
)


New https://pypi.org/project/ultralytics/8.3.136 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.127 🚀 Python-3.12.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/cad/cad.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=cad-seg-run17, nbs=64, nms=False, opset=N

train: Scanning /home/ubuntu/floor-plan-net/data/cad/labels/train-01.cache... 4336 images, 2065 backgrounds, 0 corrupt: 100%|██████████| 6401/6401 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 295.8±372.5 MB/s, size: 145.5 KB)


val: Scanning /home/ubuntu/floor-plan-net/data/cad/labels/train-00.cache... 2903 images, 857 backgrounds, 0 corrupt: 100%|██████████| 3760/3760 [00:00<?, ?it/s]


Plotting labels to runs/segment/cad-seg-run17/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.002' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000286, momentum=0.9) with parameter groups 176 weight(decay=0.0), 187 weight(decay=0.0005), 186 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/cad-seg-run17
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/20      12.3G      1.929      3.415      5.293      1.452        144        640:   0%|          | 2/401 [00:01<04:35,  1.45it/s]
Exception in thread Thread-48 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner


KeyboardInterrupt: 

    self.run()
  File "/home/ubuntu/.virtualenvs/pytorch/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/.virtualenvs/pytorch/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/home/ubuntu/.virtualenvs/pytorch/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/.virtualenvs/pytorch/lib/python3.12/site-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^

In [3]:
preds = model.predict(source="0001-0040.png", save=True)



image 1/1 /home/ubuntu/floor-plan-net/0001-0040.png: 640x640 6 single doors, 7 windows, 11 opening symbols, 4 sofas, 2 beds, 4 chairs, 3 tables, 2 tv cabinets, 1 wardrobe, 4 cabinets, 1 refrigerator, 5 airconditioners, 1 gas stove, 2 sinks, 1 washing machine, 1 toilet, 10.3ms
Speed: 6.9ms preprocess, 10.3ms inference, 18.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/cad-seg-run162


In [5]:
model.export("yolo11l-seg-cad.pt")

TypeError: Model.export() takes 1 positional argument but 2 were given

In [2]:
from ultralytics import YOLO

In [4]:
loaded_model = YOLO("models/yolo11l-seg-cad.pt")

In [5]:
preds = loaded_model.predict(source="capri_floorplan-1.jpg", save=True)



image 1/1 /home/ubuntu/floor-plan-net/capri_floorplan-1.jpg: 640x640 5 single doors, 1 double door, 2 gas stoves, 1 sink, 3 stairs, 2 elevators, 10.1ms
Speed: 5.1ms preprocess, 10.1ms inference, 230.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/predict7


In [8]:
import json

results = loaded_model('data/cad/images/train-01/0126-0009.png', save=True)  # or a list of paths

# results is a list of Results objects—loop if you passed multiple images
det_list = []
for res in results:
    for box in res.boxes:  # .boxes holds your detections
        # Access the first item of each coordinate using indexing [0]
        det_list.append({
            'class_id': int(box.cls[0]),  # Add [0] to get first element
            'x': float(box.xywh[0][0]),  # center-x
            'y': float(box.xywh[0][1]),  # center-y
            'w': float(box.xywh[0][2]),  # width
            'h': float(box.xywh[0][3]),  # height
            'confidence': float(box.conf[0])  # Add [0] to get first element
        })

# write to JSON
with open('detections.json', 'w') as f:
    json.dump(det_list, f, indent=2)


image 1/1 /home/ubuntu/floor-plan-net/data/cad/images/train-01/0126-0009.png: 640x640 4 single doors, 1 window, 3 sofas, 2 beds, 2 chairs, 2 tables, 1 tv cabinet, 3 wardrobes, 6 cabinets, 2 sinks, 1 bath, 1 washing machine, 2 toilets, 1 elevator, 74.2ms
Speed: 4.6ms preprocess, 74.2ms inference, 39.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/predict7
